In [4]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import scipy.stats as stats

In [79]:
from ta.trend import SMAIndicator
def SMA(window_para,window_para2,to_csv):
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    result.loc[result['code'] == result['code'].shift(window_para),'sma'] = SMAIndicator(close=result['close_weekly'],window=window_para).sma_indicator()
    result.loc[result['code'] == result['code'].shift(window_para2),'sma2'] = SMAIndicator(close=result['close_weekly'],window=window_para2).sma_indicator()
    
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['sma']>result['sma2']*1.01),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['sma']<result['sma2']*0.995),'signal'] = 0
    result.loc[result['code'] != result['code'].shift(1),'signal'] = 0
    result = result.ffill()
    
    result.loc[result['code'] == result['code'].shift(1),'daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    #result.to_csv('sma_raw.csv')
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checksma.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    
    
    return float(profit[-1:]), window_para, window_para2

In [80]:
SMA(13,17,to_csv = True)

MDD 0.20259703844375732


(4.069646170672616, 13, 17)

In [76]:
optimize_result = []
for window_para in range(1,20, 1):
    for window_para2 in range(2,50,5):
        x=SMA(window_para,window_para2,to_csv = False)
        optimize_result.append(x)    
optimize_result = pd.DataFrame(optimize_result)

MDD 0.4752315508994023
MDD 0.4083310780822261
MDD 0.3839206703667036
MDD 0.37900067438464546
MDD 0.39704402113488446
MDD 0.38102019251245306
MDD 0.37980635865622947
MDD 0.3582992344614066
MDD 0.3421295018803177
MDD 0.33919183755255744
MDD 0
MDD 0.29759330673473394
MDD 0.28600708002519326
MDD 0.3125129446337403
MDD 0.29699262793875103
MDD 0.3064975620732057
MDD 0.29737936746137295
MDD 0.2620188713588553
MDD 0.25765903375193444
MDD 0.2680025559643439
MDD 0.20952349689359642
MDD 0.2785884473036012
MDD 0.2662188527660413
MDD 0.2779585256821692
MDD 0.27181901766735095
MDD 0.27320560014203776
MDD 0.2505384042489539
MDD 0.24082407499752245
MDD 0.22888972210547112
MDD 0.2304048504291854
MDD 0.2104073463558746
MDD 0.293295473452809
MDD 0.267258698304885
MDD 0.26312175063060556
MDD 0.24617384387793836
MDD 0.24461972541995408
MDD 0.23345089786839843
MDD 0.22044987765534446
MDD 0.22516503588402584
MDD 0.2253160413639244
MDD 0.19677221088449978
MDD 0.271113656715629
MDD 0.24527065464632203
MDD 0.23

In [78]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1,2
123,4.069646,13,17
133,4.026448,14,17
113,3.940063,12,17
143,3.939776,15,17
93,3.781869,10,17
104,3.781472,11,22
124,3.778812,13,22
83,3.755312,9,17
106,3.746026,11,32
103,3.740827,11,17
